<a href="https://colab.research.google.com/github/AI4GoodProject/speechRecForMeeting/blob/master/CNNModelWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2,glob
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split

0.Data retrieving/some preprocessing


In [41]:
# Load the Drive helper and mount
from google.colab import drive
import pickle

# This will prompt for authorization.
drive.mount('/content/drive')

DATA_PATH = "/content/drive/My Drive/Team 6/processed-data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
!ls "/content/drive/My Drive/Team 6/processed-data"

dialogue-acts.pkl	 features.pkl	     labels.pkl
dialogue-acts-test.pkl	 features-test.pkl   labels-test.pkl
dialogue-acts-whole.pkl  features-whole.pkl


In [43]:
features= open(DATA_PATH+'/features-test.pkl','rb')
labels= open(DATA_PATH+'/labels-test.pkl','rb')
features_list = pickle.load(features)
#labels_list = pickle.load(labels)


labels_list = [random.randint(0,1) for _ in range(len(features_list))]



#print("features_list","\n", features_list[:5])
#print("length", len(features_list)) 
#print("width", len(features_list[0]))
print("-----------------")
print("labels_list","\n",labels_list[:5])
print("length", len(labels_list)) 
print("width", labels_list[0])


-----------------
labels_list 
 [0, 0, 0, 1, 1]
length 3790
width 0


In [44]:
def crossJoin (list1,list2):
  crossJoined_list = []
  
  for i in range(0,len(list1)):
    inner_list = []
    for j in range(0,1):
      inner_list.append(list1[i])
      inner_list.append(list2[i])
    crossJoined_list.append(inner_list)

  return crossJoined_list

In [45]:
# for testing purpose

list1 = np.array([['a', 'b'], ['c','d'],['e','f']])
list2 = np.array([[True], [False], [True]])

In [46]:
list3 = crossJoin (list1,list2)
list3

[[array(['a', 'b'], dtype='<U1'), array([ True])],
 [array(['c', 'd'], dtype='<U1'), array([False])],
 [array(['e', 'f'], dtype='<U1'), array([ True])]]

In [47]:
for image in features_list:
  print(image.shape)
  break

(128, 313)


In [48]:
print(features_list[0])

[[5.7969120e-04 2.8845244e-03 3.8505071e-03 ... 4.2184364e-02
  9.9059632e-03 4.5957719e-03]
 [2.2720152e-03 2.8340286e-03 3.2133576e-03 ... 3.0660164e-01
  9.4618261e-02 2.4500459e-02]
 [9.9112233e-04 4.0617207e-04 9.6240174e-04 ... 2.0881109e-01
  5.9001476e-02 3.1543978e-02]
 ...
 [8.7587335e-07 5.3553856e-07 3.8149975e-07 ... 4.6525852e-04
  2.4058839e-04 1.8763794e-04]
 [8.0240932e-07 4.7897174e-07 3.4212042e-07 ... 2.6049485e-04
  1.2429411e-04 5.5625580e-05]
 [7.1721036e-07 5.9548421e-07 5.3247766e-07 ... 1.2215057e-04
  5.6045861e-05 3.3594653e-05]]


**1. Train/validate/test split**

1.1 use np.stack to join the images into 3D tensor

1.2 add one singleton axis (you can use np.expand_dims for that) to get 4D array with channels dimension equal to 1

1.3 use train_test_split from sklearn because it allows to shuffle your data before splitting

In [63]:
#1.1 convert a list of images(np.arrays) to a 3D tensor

##?? can't directly change to tensor, cause got error of "inconsistent shape"

#convert-a-list-of-numpy-array-to-torch-tensor-list
tensor_list = [torch.from_numpy(item) for item in features_list]


tensor_list = tensor_list[:100]
labels_list = labels_list[:100]

print(torch.stack(tensor_list).shape)
print(torch.tensor(labels_list).shape)

torch.Size([100, 128, 313])
torch.Size([100])


In [50]:
#1.2 add one singleton axis (you can use np.expand_dims for that) to get 4D array with channels dimension equal to 1

##?? since we are only look into 1 channel so no need to make it 4d?
tensor = np.expand_dims(tensor_list,axis=1)

#tensor.size

/usr/local/lib/python3.7/dist-packages/numpy/lib/shape_base.py:591: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  a = asanyarray(a)
/usr/local/lib/python3.7/dist-packages/numpy/lib/shape_base.py:591: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = asanyarray(a)


In [51]:
#1.3 use train_test_split from sklearn because it allows to shuffle your data before splitting
X_train,X_test,Y_train,Y_test = train_test_split(tensor_list, labels_list, test_size=0.1, train_size=0.8, random_state=1, shuffle=True)

X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=1)
#print(len(X_test),len(X_train))
#print(len(Y_test),len(Y_train))
#print(len(X_val),len(Y_val))

In [52]:
train_data = crossJoin(X_train, Y_train)
val_data = crossJoin(X_val, Y_val)
test_data = crossJoin(X_test,Y_test)

In [53]:
print(len(train_data))
print(train_data[0])

80
[tensor([[9.6868e-03, 3.2697e-02, 3.0283e-02,  ..., 7.9474e-02, 1.7097e-01,
         1.4461e-01],
        [6.3806e-03, 1.1808e-02, 2.1545e-02,  ..., 2.6344e-01, 8.2620e-01,
         5.5500e-01],
        [2.6212e-03, 1.0034e-02, 1.7256e-02,  ..., 2.4510e-01, 3.4536e-01,
         7.3136e-01],
        ...,
        [3.0039e-07, 5.5057e-07, 1.6090e-05,  ..., 2.5093e-04, 3.3191e-03,
         8.8956e-03],
        [2.4430e-07, 5.4194e-07, 8.8940e-06,  ..., 4.4598e-04, 3.5708e-03,
         9.0889e-03],
        [2.0626e-07, 7.0101e-07, 1.7646e-05,  ..., 4.9756e-04, 2.7163e-03,
         6.9803e-03]]), 0]


In [69]:
#RuntimeError: Given groups=1, weight of size [6, 4, 5, 5], expected input[1, 2, 128, 313] to have 4 channels, but got 2 channels instead
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 4, 5)
        self.fc1 = nn.Linear(2175,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
CNN = CNN()

In [55]:
#evaluation
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(CNN.parameters(), lr=0.01, momentum=0.9)

In [72]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True)

test_dataloader = DataLoader(test_data, batch_size=4, shuffle=True)

In [74]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = CNN(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 4 == 3:    # print every 4 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 4:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     4] loss: 2.255
[1,     8] loss: 2.204
[1,    12] loss: 2.254
[1,    16] loss: 2.221
[1,    20] loss: 2.250
[2,     4] loss: 2.248
[2,     8] loss: 2.267
[2,    12] loss: 2.241
[2,    16] loss: 2.210
[2,    20] loss: 2.259
Finished Training


In [58]:
use_gpu = torch.cuda.is_available()

In [59]:
def prediction(loader, model):

  correct = 0
  total = 0
  losses = 0

  for i, (images, labels) in enumerate(loader):
    if use_gpu:
      # switch tensor type to GPU
      images = images.cuda()
      labels = labels.cuda()
    
    outputs = model(images)
    
    loss = criterion(outputs, labels)
  
    _, predictions = torch.max(outputs, 1)
  
    correct += torch.sum(labels == predictions).item()
    total += labels.shape[0]
    
    losses += loss.data.item()
    
  return losses/len(list(loader)), 1 - correct/total # we need to normalize loss with respect to the number of batches 

In [75]:
train_losses = []
test_losses = []

train_error_rates = []
test_error_rates = []


if use_gpu:
  # switch model to GPU
  CNN.cuda()

num_epochs = 5

for epoch in range(num_epochs): 
  train_loss = 0 
  n_iter = 0 
  total = 0
  correct = 0

  for i, (images, labels) in enumerate(train_dataloader): 
    optimizer.zero_grad() 

    if use_gpu: 
      images = images.cuda()
      labels = labels.cuda()

    outputs = CNN(images)
    
    # to compute the train_error_rates  
    _, predictions = torch.max(outputs, 1)
    correct += torch.sum(labels == predictions).item()
    total += labels.shape[0]
    
    # compute loss 
    loss_bs = criterion(outputs, labels)
    # compute gradients
    loss_bs.backward()
    # update weights
    optimizer.step()

    train_loss += loss_bs.detach().item()

    n_iter += 1

  train_error_rate = 1 - correct/total

  with torch.no_grad():
    test_loss, test_error_rate = prediction(test_dataloader, CNN)

  train_error_rates.append(train_error_rate)
  test_error_rates.append(test_error_rate)
  train_losses.append(train_loss/n_iter)
  test_losses.append(test_loss)

  if epoch%1 == 0:
    print('Epoch: {}/{}, Loss: {:.4f}, Error Rate: {:.1f}%'.format(epoch+1, num_epochs, train_loss/n_iter, 100*train_error_rate))

RuntimeError: ignored

In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(1,2,1)
plt.plot(train_losses, label='train', marker='o', alpha=0.7)
plt.plot(test_losses, label='test', marker='o', alpha=0.7)
plt.grid(True)
plt.legend()
plt.title('Loss')
plt.subplot(1,2,2)
plt.plot(train_error_rates, label='train', marker='o', alpha=0.7)
plt.plot(test_error_rates, label='test', marker='o', alpha=0.7)
plt.grid(True)
plt.legend()
plt.title('Error rate')
plt.show()

should follow this format
https://colab.research.google.com/drive/1HtJpNyh6oWabiVcDUG_2TJ34TtVo5JH8?authuser=2#scrollTo=D2lKHjcfscif